<a href="https://colab.research.google.com/github/abidshafee/autoML-tsModel/blob/main/AutoML_ts_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install auto_ts

     |████████████████████████████████| 97 kB 2.0 MB/s 
     |████████████████████████████████| 1.4 MB 8.0 MB/s 
     |████████████████████████████████| 1.0 MB 15.8 MB/s 
     |████████████████████████████████| 132 kB 11.6 MB/s 
     |████████████████████████████████| 9.8 MB 21.2 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: dask
    Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
